In [ ]:
#This notebook reads the xml with the gas prices and gas stations locations provided by the CRE 
#and converts them to an excel file.

In [126]:
import pandas as pd

In [131]:
import re
from bs4 import BeautifulSoup
import os

In [132]:
xml = open('places.xml','r',encoding='utf-8')

In [133]:
soup = BeautifulSoup(xml, 'lxml')

In [134]:
location_x = soup.select("x")

In [136]:
places['Longitud'] = pd.DataFrame(location_x)

In [140]:
location_y = soup.select("y")
places['Latitud'] = pd.DataFrame(location_y)

In [143]:
places = places.drop('location', axis=1)

In [194]:
places['place_id'] = places['place_id'].astype(str)

In [144]:
places.head()

,place_id,name,cre_id,Longitud,Latitud
0,2039,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",PL/658/EXP/ES/2015,-116.9214,32.47641
1,2040,"LAS MEJORES ESTACIONES, S.A DE C.V",PL/902/EXP/ES/2015,-99.74484,20.3037
2,2041,"DIAZ GAS, S.A. DE C.V.",PL/760/EXP/ES/2015,-106.4514,31.71947
3,2042,"COMBU-EXPRESS, S.A. DE C.V.",PL/825/EXP/ES/2015,-103.3042,20.71413
4,2043,"PETROMAX, S.A. DE C.V.",PL/585/EXP/ES/2015,-98.29977,26.03787


In [146]:
xml = open('prices.xml', 'r', encoding='utf-8')
soup = BeautifulSoup(xml, 'lxml')

In [147]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19874 entries, 0 to 19873
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   place_id   19874 non-null  int64  
 1   gas_price  19874 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 310.7 KB


In [150]:
prices_all = soup.select('place')

[<place place_id="11703">
 <gas_price type="regular">20.79</gas_price>
 <gas_price type="premium">22.79</gas_price>
 </place>,
 <place place_id="11702">
 <gas_price type="regular">20.05</gas_price>
 <gas_price type="premium">22.99</gas_price>
 <gas_price type="diesel">20.16</gas_price>
 </place>,
 <place place_id="11701">
 <gas_price type="regular">16.49</gas_price>
 <gas_price type="diesel">20.99</gas_price>
 </place>,
 <place place_id="11700">
 <gas_price type="regular">20.19</gas_price>
 <gas_price type="premium">21.98</gas_price>
 <gas_price type="diesel">21.65</gas_price>
 </place>,
 <place place_id="11699">
 <gas_price type="regular">20.15</gas_price>
 <gas_price type="premium">22.15</gas_price>
 </place>,
 <place place_id="11698">
 <gas_price type="regular">21.2</gas_price>
 </place>,
 <place place_id="11698">
 <gas_price type="diesel">22.3</gas_price>
 </place>,
 <place place_id="11697">
 <gas_price type="regular">19.59</gas_price>
 <gas_price type="premium">21.49</gas_price>
 

In [158]:
list_prices = []
for tag in prices_all:
    if tag.find('gas_price', type='regular') == None:
        regular = 0
    else:
        regular =  tag.find('gas_price', type='regular').text
        
    if tag.find('gas_price', type='premium') == None:
        premium = 0
    else:
        premium =  tag.find('gas_price', type='premium').text
        
    list_prices.append({'place_id': tag.get('place_id'), 'regular': regular, 'premium': premium})
    

In [161]:
prices_complete = pd.DataFrame(list_prices)

In [173]:
prices_complete['regular'] = prices_complete['regular'].astype(float)
prices_complete['premium'] = prices_complete['premium'].astype(float)

In [174]:
prices_clean = prices_complete.groupby('place_id').agg({'regular': 'max', 'premium': 'max'})

In [187]:
prices_clean = prices_clean.reset_index()

In [188]:
prices_clean.head()

,place_id,regular,premium
0,10000,21.48,23.48
1,10001,20.26,22.99
2,10002,15.87,17.60
3,10003,14.88,0.00
4,10004,19.99,21.99


In [196]:
gas_stations = pd.merge(places, prices_clean, on='place_id')

In [197]:
gas_stations.head()

,place_id,name,cre_id,Longitud,Latitud,regular,premium
0,2039,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",PL/658/EXP/ES/2015,-116.9214,32.47641,19.99,22.99
1,2040,"LAS MEJORES ESTACIONES, S.A DE C.V",PL/902/EXP/ES/2015,-99.74484,20.3037,20.53,22.05
2,2041,"DIAZ GAS, S.A. DE C.V.",PL/760/EXP/ES/2015,-106.4514,31.71947,15.19,17.86
3,2042,"COMBU-EXPRESS, S.A. DE C.V.",PL/825/EXP/ES/2015,-103.3042,20.71413,20.39,22.99
4,2043,"PETROMAX, S.A. DE C.V.",PL/585/EXP/ES/2015,-98.29977,26.03787,16.89,19.29
